<a href="https://colab.research.google.com/github/aneez-hassan/SCT_ML_4/blob/main/SCT_ML_Task4_Hand_Gesture_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe opencv-python


In [ ]:
import cv2
import numpy as np
import os

os.makedirs('/content/dataset/fist', exist_ok=True)
os.makedirs('/content/dataset/open_hand', exist_ok=True)

# Create synthetic "fist" images (filled circle)
for i in range(5):
    img = np.zeros((100, 100, 3), dtype=np.uint8)
    cv2.circle(img, (50, 50), 30, (255, 255, 255), -1)
    cv2.imwrite(f'/content/dataset/fist/fist_{i}.png', img)

# Create synthetic "open hand" images (five white lines to simulate fingers)
for i in range(5):
    img = np.zeros((100, 100, 3), dtype=np.uint8)
    for x in range(20, 81, 15):
        cv2.line(img, (x, 80), (x, 20), (255, 255, 255), 5)
    cv2.imwrite(f'/content/dataset/open_hand/open_hand_{i}.png', img)

print("Synthetic dataset with 'fist' and 'open_hand' created!")


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Paths to dataset folders
dataset_path = '/content/dataset'
gesture_folders = {
    'fist': os.path.join(dataset_path, 'fist'),
    'open_hand': os.path.join(dataset_path, 'open_hand')
}

X = []
y = []

for label, folder in enumerate(gesture_folders.values()):
    for filename in os.listdir(folder):
        if filename.endswith('.png'):
            file_path = os.path.join(folder, filename)
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img_resized = cv2.resize(img, (64, 64))
            feature_vector = img_resized.flatten()  # Flatten image to 1D array
            X.append(feature_vector)
            y.append(label)

X = np.array(X)
y = np.array(y)

print(f"Extracted features from {len(X)} images")

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Train SVM classifier
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=gesture_folders.keys()))


In [ ]:
import cv2
import os
import numpy as np

def augment_image(img):
    augmented_images = []

    # Original
    augmented_images.append(img)

    # Flip horizontally
    augmented_images.append(cv2.flip(img, 1))

    # Rotate by 15 degrees
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, 15, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h))
    augmented_images.append(rotated)

    # Rotate by -15 degrees
    M = cv2.getRotationMatrix2D(center, -15, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h))
    augmented_images.append(rotated)

    # Scale up by 10%
    scaled_up = cv2.resize(img, None, fx=1.1, fy=1.1)
    scaled_up = cv2.resize(scaled_up, (w, h))  # Resize back to original size
    augmented_images.append(scaled_up)

    # Scale down by 10%
    scaled_down = cv2.resize(img, None, fx=0.9, fy=0.9)
    scaled_down = cv2.resize(scaled_down, (w, h))
    augmented_images.append(scaled_down)

    return augmented_images

# Example usage:
dataset_path = '/content/dataset'
gesture_folders = {
    'fist': os.path.join(dataset_path, 'fist'),
    'open_hand': os.path.join(dataset_path, 'open_hand')
}

augmented_dataset_path = '/content/dataset_augmented'
os.makedirs(augmented_dataset_path, exist_ok=True)

for gesture, folder in gesture_folders.items():
    new_folder = os.path.join(augmented_dataset_path, gesture)
    os.makedirs(new_folder, exist_ok=True)

    for filename in os.listdir(folder):
        if filename.endswith('.png'):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            augmented_images = augment_image(img)
            for i, aug_img in enumerate(augmented_images):
                save_path = os.path.join(new_folder, f"{os.path.splitext(filename)[0]}_aug{i}.png")
                cv2.imwrite(save_path, aug_img)

print("Augmentation done! Augmented dataset saved at:", augmented_dataset_path)
